In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

import gc
import warnings
import datetime as dt
import math
import seaborn as sn

import tensorflow as tf # import tensorflow as a whole
from tensorflow import keras # only import Keras

# import the dense and input layer
from keras.layers import Dense, Input
# the sequential model object
from keras.models import Sequential

from numpy import linalg as LA
import math

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import tensorflow_addons as tfa

from keras.models import Sequential
from keras.layers import Input, CuDNNLSTM, Dense
from keras.layers import Embedding
from tensorflow.keras.layers import LSTM


In [2]:
#Import Data
data = pd.read_csv("data.csv")
samp_sol = pd.read_csv("sample_submission.csv")

In [3]:
data_set = pd.read_csv("data.csv")
display(data_set.head())

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [4]:
for k in data_set.columns:
    print("Column:", k)
    print(data_set[k].value_counts())
    print()

Column: id
0        1
21106    1
21104    1
21103    1
21102    1
        ..
10543    1
10542    1
10541    1
10540    1
31643    1
Name: id, Length: 31644, dtype: int64

Column: author
Thompson        17396
Janssen          2716
Weatherhead      2040
Beus             1604
Peskov           1084
Williams          891
Richenderfer      545
Mortimore         197
Kossolapov        101
Inasaka            46
Name: author, dtype: int64

Column: geometry
tube       21145
annulus     4381
plate        618
Name: geometry, dtype: int64

Column: pressure [MPa]
13.79    9226
6.89     4701
15.51    1117
10.34    1064
11.03     872
         ... 
3.77        1
12.21       1
7.06        1
7.70        1
12.14       1
Name: pressure [MPa], Length: 144, dtype: int64

Column: mass_flux [kg/m2-s]
4069.0    963
1519.0    634
1356.0    615
2034.0    533
1000.0    418
         ... 
835.0       1
967.0       1
6995.0      1
2050.0      1
1271.0      1
Name: mass_flux [kg/m2-s], Length: 733, dtype: int64

Column

In [5]:
more_data = pd.read_csv("Data_CHF_Zhao_2020_ATE.csv")

In [6]:
for k in more_data.columns:
    print("Column:", k)
    print(more_data[k].value_counts())
    print()

Column: id
1       1
1240    1
1252    1
1251    1
1250    1
       ..
619     1
618     1
617     1
616     1
1865    1
Name: id, Length: 1865, dtype: int64

Column: author
Thompson        1202
Janssen          282
Weatherhead      162
Beus              77
Williams          51
Richenderfer      36
Mortimore         19
Peskov            17
Kossolapov        12
Inasaka            7
Name: author, dtype: int64

Column: geometry
tube       1439
annulus     378
plate        48
Name: geometry, dtype: int64

Column: pressure [MPa]
13.79    610
6.89     420
11.03     59
10.34     58
15.51     52
        ... 
10.57      1
10.93      1
12.34      1
12.38      1
10.24      1
Name: pressure [MPa], Length: 114, dtype: int64

Column: mass_flux [kg/m2-s]
4069    76
1519    55
2034    42
2292    28
1533    27
        ..
903      1
852      1
783      1
4164     1
7093     1
Name: mass_flux [kg/m2-s], Length: 578, dtype: int64

Column: x_e_out [-]
 0.0886    7
 0.0145    7
 0.0990    6
-0.0187    6
-0.

In [7]:
total_data = data_set.append(more_data).drop("id", axis=1)

/var/folders/tr/3zw9td7531z0_hhwc8105rv00000gn/T/ipykernel_7360/3739133671.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_data = data_set.append(more_data).drop("id", axis=1)


In [8]:
dummies_author = pd.get_dummies(total_data.author)
merged_with_authors = pd.concat([total_data, dummies_author], axis='columns')
merged_with_authors = merged_with_authors.drop(['author'], axis='columns')

dummies_geometry = pd.get_dummies(merged_with_authors.geometry)
merged_with_authors_and_geometry = pd.concat([merged_with_authors, dummies_geometry], axis='columns')
finalized_data = merged_with_authors_and_geometry.drop(['geometry'], axis='columns')

In [9]:
final_test = finalized_data[pd.isnull(finalized_data["x_e_out [-]"])]
final_test = final_test.fillna(0)
final_test = final_test.drop("x_e_out [-]", axis=1)
finalized_data = finalized_data.dropna(subset=["x_e_out [-]"])
finalized_data = finalized_data.fillna(0)

In [10]:
submission_indices = list(final_test.index)

## Train / Test Split

In [11]:
X = finalized_data.drop("x_e_out [-]", axis=1)
y = finalized_data["x_e_out [-]"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
def convert_to_sequences(df):
    
    m,n = df.shape
    
    sequences = np.zeros((m, 2*n))
    
    i = 0
    for index, row in df.iterrows():
        
        k = 1
        j = 0
        for item in row: 
            if item != 0:
                sequences[i, j] = k
                sequences[i, j + 1] = item
                j += 2
            k += 1
        
        i+= 1

    return sequences 

In [14]:
X_train = convert_to_sequences(X_train)
X_test = convert_to_sequences(X_test)

In [15]:
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [16]:
X_train.shape

(15472, 1, 38)

## Construct Model

In [17]:
model = Sequential()
model.add(LSTM(19, input_shape=(1,2*19)))
model.add(Dense(1))

2023-05-27 11:56:17.194720: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
model.compile(loss=['mean_squared_error'] , optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 19)                4408      
                                                                 
 dense (Dense)               (None, 1)                 20        
                                                                 
Total params: 4,428
Trainable params: 4,428
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, epochs=100, batch_size=20, verbose=2)

Epoch 1/100
774/774 - 1s - loss: 0.0202 - accuracy: 2.5853e-04 - 1s/epoch - 2ms/step
Epoch 2/100
774/774 - 1s - loss: 0.0122 - accuracy: 2.5853e-04 - 545ms/epoch - 704us/step
Epoch 3/100
774/774 - 1s - loss: 0.0115 - accuracy: 2.5853e-04 - 543ms/epoch - 701us/step
Epoch 4/100
774/774 - 1s - loss: 0.0108 - accuracy: 2.5853e-04 - 542ms/epoch - 700us/step
Epoch 5/100
774/774 - 1s - loss: 0.0104 - accuracy: 2.5853e-04 - 541ms/epoch - 699us/step
Epoch 6/100
774/774 - 1s - loss: 0.0100 - accuracy: 2.5853e-04 - 541ms/epoch - 698us/step
Epoch 7/100
774/774 - 1s - loss: 0.0100 - accuracy: 2.5853e-04 - 539ms/epoch - 696us/step
Epoch 8/100
774/774 - 1s - loss: 0.0101 - accuracy: 2.5853e-04 - 537ms/epoch - 694us/step
Epoch 9/100
774/774 - 1s - loss: 0.0099 - accuracy: 2.5853e-04 - 538ms/epoch - 695us/step
Epoch 10/100
774/774 - 1s - loss: 0.0099 - accuracy: 2.5853e-04 - 539ms/epoch - 696us/step
Epoch 11/100
774/774 - 1s - loss: 0.0098 - accuracy: 2.5853e-04 - 538ms/epoch - 695us/step
Epoch 12/100


Epoch 92/100
774/774 - 1s - loss: 0.0092 - accuracy: 2.5853e-04 - 557ms/epoch - 720us/step
Epoch 93/100
774/774 - 1s - loss: 0.0091 - accuracy: 2.5853e-04 - 604ms/epoch - 781us/step
Epoch 94/100
774/774 - 1s - loss: 0.0092 - accuracy: 2.5853e-04 - 569ms/epoch - 736us/step
Epoch 95/100
774/774 - 1s - loss: 0.0091 - accuracy: 2.5853e-04 - 548ms/epoch - 707us/step
Epoch 96/100
774/774 - 1s - loss: 0.0094 - accuracy: 2.5853e-04 - 577ms/epoch - 745us/step
Epoch 97/100
774/774 - 1s - loss: 0.0092 - accuracy: 2.5853e-04 - 568ms/epoch - 734us/step
Epoch 98/100
774/774 - 1s - loss: 0.0091 - accuracy: 2.5853e-04 - 546ms/epoch - 705us/step
Epoch 99/100
774/774 - 1s - loss: 0.0091 - accuracy: 2.5853e-04 - 555ms/epoch - 717us/step
Epoch 100/100
774/774 - 1s - loss: 0.0090 - accuracy: 2.5853e-04 - 545ms/epoch - 704us/step


In [20]:
y_hat = model.predict(X_test)

239/239 [==============================] - 0s 378us/step


In [26]:
math.sqrt(np.mean(np.square(y_test - y_hat)))# / math.sqrt(np.mean(np.square(y_test)))

0.10882202854664608

# Perform Prediction

In [27]:
X_submission = convert_to_sequences(final_test)
X_submission = np.expand_dims(X_submission, axis=1)

In [28]:
y_submission = model.predict(X_submission)

326/326 [==============================] - 0s 391us/step


In [29]:
y_submission = list(y_submission.reshape(len(y_submission)))

In [30]:
df = pd.DataFrame(data={"id": submission_indices, "x_e_out [-]": y_submission})
df.to_csv("submission.csv", sep=',',index=False)